In [1]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    from io import StringIO
    import time

    def get_day_price(code, max_pages=720):
        base_url = f"https://finance.naver.com/item/sise_day.naver?code={code}"
        headers = {"User-Agent": "Mozilla/5.0"}
        df_total = pd.DataFrame()

        for page in range(1, max_pages + 1):
            url = f"{base_url}&page={page}"
            res = requests.get(url, headers=headers)
            if res.status_code != 200:
                print(f"페이지 {page} 요청 실패: 상태 코드 {res.status_code}")
                break

            soup = BeautifulSoup(res.text, "html.parser")
            table = soup.find("table", class_="type2")
            if table is None:
                print(f"페이지 {page}에서 테이블을 찾을 수 없습니다.")
                break

            df = pd.read_html(StringIO(str(table)), header=0)[0]
            df = df.dropna()

            # 👉 만약 페이지에 데이터가 없는 경우 자동 종료
            if df.empty:
                print(f"\n✅ 데이터 없음: 페이지 {page}에서 종료합니다.")
                break

            # 전일비 처리
            def parse_change(val):
                val = str(val).strip().replace(",", "")
                if val.startswith("상승"):
                    return int(val.replace("상승", "").strip())
                elif val.startswith("하락"):
                    return -int(val.replace("하락", "").strip())
                elif val.startswith("보합"):
                    return 0
                else:
                    return None

            df["전일비"] = df["전일비"].apply(parse_change)

            # 거래량 처리
            df["거래량"] = df["거래량"].astype(str).str.replace(",", "")
            df["거래량"] = df["거래량"].apply(lambda x: int(float(x)))

            # ✅ 출력 – 첫 페이지만 헤더 포함
            if page == 1:
                print(df.to_string(index=False))  # 헤더 포함
            else:
                print(df.to_string(index=False, header=False))  # 헤더 생략

            df_total = pd.concat([df_total, df], ignore_index=True)
            time.sleep(0.5)

        return df_total

    # 종목 코드 및 기업명 설정
    code = "010140"
    company_name = "삼성중공업"

    # 데이터 수집
    df = get_day_price(code, max_pages=720)

    # 컬럼명 정리
    df.columns = ["날짜", "종가", "전일비", "시가", "고가", "저가", "거래량"]

    # 날짜 형식 변환
    df["날짜"] = pd.to_datetime(df["날짜"])

    # ✅ 날짜 기준으로 중복 제거
    df = df.drop_duplicates(subset="날짜", keep="first")

    # 날짜 내림차순 정렬
    df = df.sort_values("날짜", ascending=False)

    # CSV 파일로 저장
    filename = f"{company_name}.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\n✅ {filename} 파일이 저장되었습니다.")


        날짜      종가  전일비      시가      고가      저가      거래량
2025.04.25 14720.0  130 14500.0 14900.0 14270.0 14117412
2025.04.24 14590.0  290 14310.0 14600.0 14020.0  6597321
2025.04.23 14300.0  200 14210.0 14330.0 14070.0  3820358
2025.04.22 14100.0  110 13840.0 14180.0 13840.0  4738010
2025.04.21 13990.0 -530 14520.0 14530.0 13920.0  9035695
2025.04.18 14520.0 -380 15010.0 15150.0 14370.0  7886520
2025.04.17 14900.0  410 14570.0 14900.0 14560.0  6465940
2025.04.16 14490.0  -50 14550.0 14830.0 14370.0  5931178
2025.04.15 14540.0   90 14550.0 14620.0 14400.0  4038501
2025.04.14 14450.0  -50 14810.0 14820.0 14370.0  6055197
2025.04.11 14500.0   550 14150.0 14520.0 14090.0 10701486
2025.04.10 13950.0   910 13790.0 13990.0 13540.0  8268063
2025.04.09 13040.0  -180 13530.0 13610.0 12930.0  7249419
2025.04.08 13220.0   500 13040.0 13350.0 12780.0  7087137
2025.04.07 12720.0 -1000 13170.0 13180.0 12650.0  7764209
2025.04.04 13720.0  -290 13790.0 14230.0 13380.0  9496140
2025.04.03 14010.0   130 